In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import random
import os

In [2]:
print(tf.__version__)

2.8.0


In [3]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
train_data = pd.read_csv("dataset/image_classification/plant_pathology/train.csv")
train_data.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0


In [5]:
img_path = "dataset/image_classification/plant_pathology/images"

In [6]:
train_data["image_id"].to_numpy()

array(['Train_0', 'Train_1', 'Train_2', ..., 'Train_1818', 'Train_1819',
       'Train_1820'], dtype=object)

In [7]:
img_path = [os.path.join(img_path, filename + ".jpg") for filename in train_data["image_id"].to_numpy()]

In [8]:
img_path[0]

'dataset/image_classification/plant_pathology/images\\Train_0.jpg'

In [9]:
img_targets = train_data.drop("image_id", axis=1).to_numpy()
img_targets[0]

array([0, 0, 0, 1], dtype=int64)

In [10]:
IMG_SIZE = 128
def load_and_preprocess_image(image_path, label, img_size=IMG_SIZE):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, size=[img_size, img_size])
    return image, label

In [11]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomHeight(0.2),
    layers.RandomWidth(0.2),
    layers.Resizing(128, 128)
], name="data_augmentation_layer")

In [12]:
image = load_and_preprocess_image(img_path[0], img_targets[0])
image

(<tf.Tensor: shape=(128, 128, 3), dtype=float32, numpy=
 array([[[0.13333334, 0.20392159, 0.05490196],
         [0.13725491, 0.21960786, 0.05882353],
         [0.15294118, 0.22352943, 0.05882353],
         ...,
         [0.1627451 , 0.20588237, 0.08039216],
         [0.19869794, 0.24183519, 0.10850184],
         [0.18594517, 0.22908244, 0.09574909]],
 
        [[0.12646294, 0.19316025, 0.0460861 ],
         [0.1323606 , 0.21471356, 0.05392923],
         [0.14997703, 0.22056527, 0.05585938],
         ...,
         [0.15292586, 0.19606313, 0.07446385],
         [0.19312961, 0.23237593, 0.10487899],
         [0.18826595, 0.2314032 , 0.10001533]],
 
        [[0.1137255 , 0.19607845, 0.04313726],
         [0.12941177, 0.21176472, 0.0509804 ],
         [0.14509805, 0.21568629, 0.0509804 ],
         ...,
         [0.13725491, 0.20392159, 0.07843138],
         [0.1467448 , 0.20556834, 0.07615656],
         [0.15717678, 0.22776502, 0.07090227]],
 
        ...,
 
        [[0.7117647 , 0.6921569 

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
train_img, val_img, train_labels, val_labels = train_test_split(img_path, 
                                                                img_targets, 
                                                                test_size=0.2,
                                                                random_state=42)

In [15]:
len(train_img), len(val_img), len(train_labels), len(val_labels)

(1456, 365, 1456, 365)

In [16]:
train_img[0]

'dataset/image_classification/plant_pathology/images\\Train_1455.jpg'

In [17]:
train_labels[0]

array([0, 0, 1, 0], dtype=int64)

In [18]:
pd.DataFrame(val_labels).value_counts()

0  1  2  3
0  0  0  1    127
      1  0    120
1  0  0  0    100
0  1  0  0     18
Name: count, dtype: int64

In [19]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_img, train_labels))
train_dataset = train_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
train_dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.int64, name=None))>

In [20]:
val_dataset = tf.data.Dataset.from_tensor_slices((val_img, val_labels))
val_dataset = val_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
val_dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.int64, name=None))>

In [21]:
len(train_dataset), len(val_dataset)

(46, 12)

In [22]:
INPUT_SHAPE = (128, 128, 3)

In [22]:
tf.keras.backend.clear_session()

inputs = layers.Input(shape=INPUT_SHAPE, dtype=tf.float32)
x = layers.Conv2D(10, 3, activation="relu")(inputs)
x = layers.Conv2D(10, 3, activation="relu")(x)
x = layers.MaxPool2D(2)(x)
x = layers.Conv2D(10, 3, activation="relu")(x)
x = layers.Conv2D(10, 3, activation="relu")(x)
x = layers.MaxPool2D(2)(x)
x = layers.Flatten()(x)
outputs = layers.Dense(4, activation="softmax")(x)

model_1 = tf.keras.Model(inputs, outputs, name="model_1")

In [23]:
model_1.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 222, 222, 10)      280       
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 10)      910       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 10)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 108, 108, 10)      910       
                                                                 
 conv2d_3 (Conv2D)           (None, 106, 106, 10)      910       
                                                           

In [24]:
model_1_init_weights = model_1.get_weights()

In [25]:
model_1.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [26]:
model_1_history = model_1.fit(train_dataset,
                              validation_data=val_dataset,
                              epochs=5,
                              validation_steps=int(len(val_dataset) * 0.1))

Epoch 1/5
46/46 [==============================] - 27s 539ms/step - loss: 1.2829 - accuracy: 0.3242 - val_loss: 1.2121 - val_accuracy: 0.5000
Epoch 2/5
46/46 [==============================] - 26s 568ms/step - loss: 1.2219 - accuracy: 0.4107 - val_loss: 1.1791 - val_accuracy: 0.4375
Epoch 3/5
46/46 [==============================] - 26s 570ms/step - loss: 1.1528 - accuracy: 0.4643 - val_loss: 1.1298 - val_accuracy: 0.4375
Epoch 4/5
46/46 [==============================] - 27s 576ms/step - loss: 0.9947 - accuracy: 0.5749 - val_loss: 1.2165 - val_accuracy: 0.4375
Epoch 5/5
46/46 [==============================] - 27s 574ms/step - loss: 0.7424 - accuracy: 0.7067 - val_loss: 1.4582 - val_accuracy: 0.4375


In [27]:
model_1.evaluate(val_dataset)

12/12 [==============================] - 7s 524ms/step - loss: 1.4151 - accuracy: 0.4384


[1.4150738716125488, 0.4383561611175537]

In [28]:
model_1.evaluate(train_dataset)

46/46 [==============================] - 26s 553ms/step - loss: 0.5141 - accuracy: 0.8029


[0.5140912532806396, 0.8028846383094788]

In [29]:
tf.keras.backend.clear_session()

In [30]:
inputs = layers.Input(shape=INPUT_SHAPE, dtype=tf.float32)
x = data_augmentation(inputs)
x = layers.Conv2D(10, 3, activation="relu")(x)
x = layers.Conv2D(10, 3, activation="relu")(x)
x = layers.MaxPool2D(2)(x)
x = layers.Conv2D(10, 3, activation="relu")(x)
x = layers.Conv2D(10, 3, activation="relu")(x)
x = layers.MaxPool2D(2)(x)
x = layers.Flatten()(x)
outputs = layers.Dense(4, activation="softmax")(x)

model_2 = tf.keras.Model(inputs, outputs, name="model_2")

In [31]:
model_2.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation_layer (Se  (None, 224, 224, 3)      0         
 quential)                                                       
                                                                 
 conv2d (Conv2D)             (None, 222, 222, 10)      280       
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 10)      910       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 10)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 108, 108, 10)      910 

In [32]:
model_2.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_2_history = model_2.fit(train_dataset,
                              validation_data=val_dataset,
                              validation_steps=int(len(val_dataset) * 0.1),
                              epochs=5)

Epoch 1/5
46/46 [==============================] - 28s 582ms/step - loss: 1.2468 - accuracy: 0.3956 - val_loss: 1.1438 - val_accuracy: 0.7500
Epoch 2/5
46/46 [==============================] - 29s 617ms/step - loss: 1.0815 - accuracy: 0.5742 - val_loss: 0.8433 - val_accuracy: 0.7500
Epoch 3/5
46/46 [==============================] - 28s 599ms/step - loss: 0.9645 - accuracy: 0.6319 - val_loss: 0.8160 - val_accuracy: 0.6562
Epoch 4/5
46/46 [==============================] - 28s 603ms/step - loss: 0.9057 - accuracy: 0.6717 - val_loss: 0.7366 - val_accuracy: 0.6875
Epoch 5/5
46/46 [==============================] - 28s 611ms/step - loss: 0.8275 - accuracy: 0.6930 - val_loss: 0.6847 - val_accuracy: 0.7500


In [33]:
model_2.evaluate(train_dataset), model_2.evaluate(val_dataset)

12/12 [==============================] - 7s 550ms/step - loss: 0.8299 - accuracy: 0.6795


([0.7703270316123962, 0.7197802066802979],
 [0.8299142718315125, 0.6794520616531372])

In [37]:
inputs = layers.Input(shape=INPUT_SHAPE, dtype=tf.float32)
x = data_augmentation(inputs)
x = layers.Conv2D(32, 3, activation="relu")(x)
x = layers.Conv2D(32, 3, activation="relu")(x)
x = layers.MaxPool2D(2)(x)
x = layers.Conv2D(32, 3, activation="relu")(x)
x = layers.Conv2D(32, 3, activation="relu")(x)
x = layers.MaxPool2D(2)(x)
x = layers.Flatten()(x)
outputs = layers.Dense(4, activation="softmax")(x)

model_3 = tf.keras.Model(inputs, outputs, name="model_3")

In [38]:
model_3.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation_layer (Se  (None, 224, 224, 3)      0         
 quential)                                                       
                                                                 
 conv2d_8 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 conv2d_9 (Conv2D)           (None, 220, 220, 32)      9248      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 110, 110, 32)     0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 108, 108, 32)      9248

In [39]:
model_3.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_3_history = model_3.fit(train_dataset,
                              validation_data=val_dataset,
                              validation_steps=int(len(val_dataset) * 0.1),
                              epochs=5)

Epoch 1/5
46/46 [==============================] - 30s 589ms/step - loss: 1.2660 - accuracy: 0.3242 - val_loss: 1.2072 - val_accuracy: 0.3750
Epoch 2/5
46/46 [==============================] - 28s 598ms/step - loss: 1.2429 - accuracy: 0.3537 - val_loss: 1.1793 - val_accuracy: 0.3125
Epoch 3/5
46/46 [==============================] - 27s 575ms/step - loss: 1.2165 - accuracy: 0.3956 - val_loss: 1.1417 - val_accuracy: 0.4062
Epoch 4/5
46/46 [==============================] - 26s 567ms/step - loss: 1.2036 - accuracy: 0.4093 - val_loss: 1.1237 - val_accuracy: 0.3750
Epoch 5/5
46/46 [==============================] - 28s 603ms/step - loss: 1.2215 - accuracy: 0.4045 - val_loss: 1.2142 - val_accuracy: 0.3750


In [87]:
tf.keras.backend.clear_session()

inputs = layers.Input(shape=INPUT_SHAPE, dtype=tf.float32)
x = data_augmentation(inputs)
x = layers.Conv2D(8, 3, activation="relu")(x)
x = layers.Conv2D(8, 3, activation="relu")(x)
x = layers.MaxPool2D(2)(x)
x = layers.Conv2D(8, 3, activation="relu")(x)
x = layers.Conv2D(8, 3, activation="relu")(x)
x = layers.MaxPool2D(2)(x)
x = layers.Flatten()(x)
outputs = layers.Dense(4, activation="softmax")(x)

model_4 = tf.keras.Model(inputs, outputs, name="model_4")

model_4_init_weights = model_4.get_weights()

In [88]:
model_4.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation_layer (Se  (None, 224, 224, 3)      0         
 quential)                                                       
                                                                 
 conv2d (Conv2D)             (None, 222, 222, 8)       224       
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 8)       584       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 8)      0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 108, 108, 8)       584 

In [90]:
model_4.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_4_history = model_4.fit(train_dataset,
                              validation_data=val_dataset,
                              validation_steps=int(len(val_dataset) * 0.1),
                              epochs=5)

Epoch 1/5
46/46 [==============================] - 28s 583ms/step - loss: 1.2748 - accuracy: 0.3674 - val_loss: 1.2453 - val_accuracy: 0.3438
Epoch 2/5
46/46 [==============================] - 26s 569ms/step - loss: 1.2348 - accuracy: 0.3620 - val_loss: 1.2165 - val_accuracy: 0.4062
Epoch 3/5
46/46 [==============================] - 26s 568ms/step - loss: 1.2186 - accuracy: 0.3846 - val_loss: 1.1488 - val_accuracy: 0.4375
Epoch 4/5
46/46 [==============================] - 28s 595ms/step - loss: 1.1935 - accuracy: 0.4176 - val_loss: 1.1377 - val_accuracy: 0.5000
Epoch 5/5
46/46 [==============================] - 28s 597ms/step - loss: 1.1714 - accuracy: 0.4663 - val_loss: 1.1057 - val_accuracy: 0.5312


In [91]:
model_4.evaluate(train_dataset), model_4.evaluate(val_dataset)

12/12 [==============================] - 7s 531ms/step - loss: 1.1402 - accuracy: 0.4932


([1.1449439525604248, 0.4862637221813202],
 [1.1401708126068115, 0.4931506812572479])

In [128]:
tf.keras.backend.clear_session()

inputs = layers.Input(shape=INPUT_SHAPE, dtype=tf.float32)
x = data_augmentation(inputs, training=True)
x = layers.Conv2D(64, 3, activation="relu")(x)
x = layers.MaxPool2D(2)(x)
x = layers.Conv2D(64, 3, activation="relu")(x)
x = layers.MaxPool2D(2)(x)
x = layers.Conv2D(128, 3, activation="relu")(x)
x = layers.MaxPool2D(2)(x)
x = layers.Conv2D(128, 3, activation="relu")(x)
x = layers.MaxPool2D(2)(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512, activation="relu")(x)
outputs = layers.Dense(4, activation="softmax")(x)

model_5 = tf.keras.Model(inputs, outputs, name="model_5")

In [129]:
model_5.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 data_augmentation_layer (Se  (None, 128, 128, 3)      0         
 quential)                                                       
                                                                 
 conv2d (Conv2D)             (None, 126, 126, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0   

In [ ]:
model_5.compile(loss='categorical_crossentropy', 
                optimizer='rmsprop', 
                metrics=['accuracy'])

In [132]:
model_5_history = model_5.fit(train_dataset,
                              validation_data=val_dataset,
                              epochs=50,
                              callbacks=[tensorboard(model_5.name), 
                                         early_stopping, 
                                         checkpoint])

Epoch 1/50
46/46 [==============================] - 32s 673ms/step - loss: 1.3674 - accuracy: 0.3194 - val_loss: 1.3022 - val_accuracy: 0.2740
Epoch 2/50
46/46 [==============================] - 31s 679ms/step - loss: 1.2654 - accuracy: 0.3537 - val_loss: 1.3476 - val_accuracy: 0.3836
Epoch 3/50
46/46 [==============================] - 32s 686ms/step - loss: 1.1932 - accuracy: 0.4457 - val_loss: 1.1900 - val_accuracy: 0.5068
Epoch 4/50
46/46 [==============================] - 31s 683ms/step - loss: 1.1074 - accuracy: 0.5275 - val_loss: 0.9054 - val_accuracy: 0.6630
Epoch 5/50
46/46 [==============================] - 32s 685ms/step - loss: 0.9030 - accuracy: 0.6435 - val_loss: 0.9628 - val_accuracy: 0.5863
Epoch 6/50
46/46 [==============================] - 31s 676ms/step - loss: 0.8170 - accuracy: 0.6552 - val_loss: 0.7480 - val_accuracy: 0.6767
Epoch 7/50
46/46 [==============================] - 31s 675ms/step - loss: 0.7555 - accuracy: 0.6909 - val_loss: 1.0459 - val_accuracy: 0.6274

In [133]:
model_5.evaluate(train_dataset), model_5.evaluate(val_dataset)

12/12 [==============================] - 7s 541ms/step - loss: 0.9018 - accuracy: 0.6384


([0.8454076051712036, 0.6435439586639404],
 [0.9017753601074219, 0.6383561491966248])

In [134]:
%load_ext tensorboard
%tensorboard --logdir model_logs/plant_pathology --port 8085

In [135]:
model_5_loaded = tf.keras.models.load_model("plant_pathology_model_5.h5")
model_5_loaded.evaluate(train_dataset), model_5_loaded.evaluate(val_dataset)

12/12 [==============================] - 7s 531ms/step - loss: 0.6140 - accuracy: 0.7699


([0.5817788243293762, 0.7870879173278809],
 [0.6140056848526001, 0.7698630094528198])

In [23]:
tf.keras.backend.clear_session()

inputs = layers.Input(shape=INPUT_SHAPE, dtype=tf.float32)
x = data_augmentation(inputs, training=True)
x = layers.Conv2D(64, 3, activation="relu")(x)
x = layers.MaxPool2D(2)(x)
x = layers.Conv2D(64, 3, activation="relu")(x)
x = layers.MaxPool2D(2)(x)
x = layers.Conv2D(128, 3, activation="relu")(x)
x = layers.MaxPool2D(2)(x)
x = layers.Conv2D(128, 3, activation="relu")(x)
x = layers.MaxPool2D(2)(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation="relu")(x)
outputs = layers.Dense(4, activation="softmax")(x)

model_6 = tf.keras.Model(inputs, outputs, name="model_6")

In [24]:
model_6.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 data_augmentation_layer (Se  (None, 128, 128, 3)      0         
 quential)                                                       
                                                                 
 conv2d (Conv2D)             (None, 126, 126, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0   

In [26]:
model_6.compile(loss='categorical_crossentropy', 
                optimizer='rmsprop', 
                metrics=['accuracy'])

In [32]:
import datetime

def tensorboard(model_name):
    return tf.keras.callbacks.TensorBoard(os.path.join("model_logs/plant_pathology", model_name, datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))
    
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)

checkpoint = tf.keras.callbacks.ModelCheckpoint("h5_models/plant_pathology_model_7.h5", save_best_only=True)

In [29]:
model_6_history = model_6.fit(train_dataset,
                              validation_data=val_dataset,
                              epochs=50,
                              callbacks=[tensorboard(model_6.name), 
                                         early_stopping, 
                                         checkpoint])

Epoch 1/50
46/46 [==============================] - 47s 818ms/step - loss: 1.3251 - accuracy: 0.3187 - val_loss: 1.3396 - val_accuracy: 0.2740
Epoch 2/50
46/46 [==============================] - 37s 812ms/step - loss: 1.2607 - accuracy: 0.3365 - val_loss: 1.2813 - val_accuracy: 0.2740
Epoch 3/50
46/46 [==============================] - 38s 817ms/step - loss: 1.2500 - accuracy: 0.3523 - val_loss: 1.2512 - val_accuracy: 0.3781
Epoch 4/50
46/46 [==============================] - 37s 807ms/step - loss: 1.1900 - accuracy: 0.4554 - val_loss: 1.5773 - val_accuracy: 0.3616
Epoch 5/50
46/46 [==============================] - 38s 814ms/step - loss: 0.9851 - accuracy: 0.5982 - val_loss: 1.1316 - val_accuracy: 0.5123
Epoch 6/50
46/46 [==============================] - 38s 815ms/step - loss: 0.8482 - accuracy: 0.6593 - val_loss: 0.9663 - val_accuracy: 0.5589
Epoch 7/50
46/46 [==============================] - 37s 812ms/step - loss: 0.7968 - accuracy: 0.6786 - val_loss: 0.6887 - val_accuracy: 0.7205

In [30]:
tf.keras.backend.clear_session()

inputs = layers.Input(shape=INPUT_SHAPE, dtype=tf.float32)
x = data_augmentation(inputs, training=True)
x = layers.Conv2D(128, 3, activation="relu")(x)
x = layers.MaxPool2D(2)(x)
x = layers.Conv2D(128, 3, activation="relu")(x)
x = layers.MaxPool2D(2)(x)
x = layers.Conv2D(128, 3, activation="relu")(x)
x = layers.MaxPool2D(2)(x)
x = layers.Conv2D(128, 3, activation="relu")(x)
x = layers.MaxPool2D(2)(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation="relu")(x)
outputs = layers.Dense(4, activation="softmax")(x)

model_7 = tf.keras.Model(inputs, outputs, name="model_7")

In [31]:
model_7.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 data_augmentation_layer (Se  (None, 128, 128, 3)      0         
 quential)                                                       
                                                                 
 conv2d (Conv2D)             (None, 126, 126, 128)     3584      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 128)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 128)      0   

In [34]:
model_7.compile(loss='categorical_crossentropy', 
                optimizer='rmsprop', 
                metrics=['accuracy'])

model_7_history = model_7.fit(train_dataset,
                              validation_data=val_dataset,
                              epochs=50,
                              callbacks=[tensorboard(model_7.name), 
                                         early_stopping, 
                                         checkpoint])

Epoch 1/50
46/46 [==============================] - 34s 666ms/step - loss: 1.4628 - accuracy: 0.3262 - val_loss: 1.3845 - val_accuracy: 0.2740
Epoch 2/50
46/46 [==============================] - 31s 669ms/step - loss: 1.2699 - accuracy: 0.3427 - val_loss: 1.2817 - val_accuracy: 0.2740
Epoch 3/50
46/46 [==============================] - 31s 659ms/step - loss: 1.2576 - accuracy: 0.3599 - val_loss: 1.2136 - val_accuracy: 0.4438
Epoch 4/50
46/46 [==============================] - 33s 721ms/step - loss: 1.2122 - accuracy: 0.4093 - val_loss: 1.3825 - val_accuracy: 0.4027
Epoch 5/50
46/46 [==============================] - 34s 740ms/step - loss: 1.1022 - accuracy: 0.5185 - val_loss: 1.0023 - val_accuracy: 0.6137
Epoch 6/50
46/46 [==============================] - 34s 737ms/step - loss: 1.0252 - accuracy: 0.5900 - val_loss: 1.2591 - val_accuracy: 0.5288
Epoch 7/50
46/46 [==============================] - 33s 720ms/step - loss: 0.8744 - accuracy: 0.6593 - val_loss: 0.7995 - val_accuracy: 0.6493